### Группировка по вопросам (красиво, но трудно обрабатывать)

In [858]:
import pandas as pd

In [871]:
pd.set_option('display.max_rows', 10)

In [872]:
df_all = pd.read_csv('../data/df_all.csv').drop(['Unnamed: 0', 'id', 'cluster', 'sentiment'], axis=1)

In [873]:
df_all

,question,answer,count
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0
...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0


In [874]:
index = pd.MultiIndex.from_arrays(df_all.iloc[:, :-1].values.T, names=('question', 'answer'))
df_grouped = pd.DataFrame({'count' : df_all.iloc[:, -1:].values.T[0]}, index=index)

In [875]:
df_grouped

count
question                                     answer                               
Что вы сможете использовать в своей работе?  формулировать проблему, 5 поче    1.0
                                             поиск потерь                      1.0
                                             правило 5 «почему» :)             1.0
                                             умение работать в команде         1.0
                                             картирование и анализ             1.0
...                                                                            ...
•Где вы хотите провести следующую HR-сессию? в москве                          1.0
                                             в сочи                            1.0
                                             сочи                              1.0
                                             окбм                              1.0
                                             сочи                              1.0

[29664 rows x 1 columns]

## Преобразование аббревиатур и сокращений

In [876]:
def get_abbreviations(f):
    abbreviations = dict()
    with open(f) as file:
        for line in file.readlines():
            key, value = map(str.strip, line.split('->'))
            abbreviations[key] = value
    return abbreviations

In [877]:
import re

In [878]:
def replace_abbreviations(sentence):
    abbreviations = get_abbreviations('../data/сокращения.txt')
    anglicisms = get_abbreviations('../data/англицизмы.txt')
    all_bad_words = abbreviations | anglicisms
    words = sentence.lower().split()
    for i in range(len(words)):
        if words[i] in all_bad_words:
            words[i] = all_bad_words[words[i]]
    return ' '.join(words)

In [879]:
replace_abbreviations('я все раб. время в it отдыхаю, чил, зе бест. бест!')

'я все работа время в информационные технологии отдыхаю, чил, зе бест. бест!'

## Определение эмоциональной окраски

In [880]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [881]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [882]:
def get_keys(list_of_dicts):
    keys = []
    for d in list_of_dicts:
        for key in d:
            keys.append(key)
    return keys

In [883]:
# до преобразования аббревиатур и сокращений

In [884]:
messages = df_all['answer'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [885]:
df_all['corrected'] = list(map(lambda answer: replace_abbreviations(answer), df_all['answer'].values))

In [886]:
df_all[df_all['sentiment'] == 'skip']

,question,answer,count,sentiment,corrected
132,"Что для вас ""культура доверия"" в организации?",отсутствие наказаний за ошибки,1.0,skip,отсутствие наказаний за ошибки
198,Что хорошего сделал за день?,макет,1.0,skip,макет
360,Ваша главная эмоции сейчас?,страшно чутка,1.0,skip,страшно немного
449,"Считаете ли вы, что ТТТ могут помочь?","да, но…",1.0,skip,"да, но…"
451,"Считаете ли вы, что ТТТ могут помочь?","да, но",1.0,skip,"да, но"
...,...,...,...,...,...
29515,Кто вы по профессии?,ведьма,1.0,skip,ведьма
29518,Кто вы по профессии?,менеджер,1.0,skip,менеджер
29529,Кто я?,робота,1.0,skip,робота
29535,Кто я?,харошо,1.0,skip,харошо


In [887]:
# до преобразования аббревиатур и сокращений

In [888]:
messages = df_all['corrected'].values
list_of_sentiments = model.predict(messages, k=1)
df_all['sentiment_corr'] = [sentiment if sentiment != 'speech' else 'positive' for sentiment in get_keys(list_of_sentiments)]

In [889]:
df_all[df_all['sentiment'] != df_all['sentiment_corr']][['question','answer','sentiment','corrected','sentiment_corr']]

,question,answer,sentiment,corrected,sentiment_corr
360,Ваша главная эмоции сейчас?,страшно чутка,skip,страшно немного,negative
520,Какие лучшие практики мотивации рабочих и ИТР ...,стабильная и достойная з/п,negative,стабильная и достойная зарплата,neutral
1458,Как ваше настроение?,#росатомсила,skip,крутая госкорпорация росатом сила,positive
1593,Как ваше настроение?,гуд,neutral,хорошо,positive
1622,Как ваше настроение?,бамбалейла,skip,отлично,positive
...,...,...,...,...,...
29129,Какой ваш девиз,ВК - от Росатома пароль,skip,внешний контроль - от крутая госкорпорация рос...,neutral
29133,Какой ваш девиз,"ВК, к лидерству Росатома парол",skip,"вк, к лидерству крутая госкорпорация росатом п...",positive
29134,Какой ваш девиз,ВК к лидр Росатома пароль,skip,внешний контроль к лидерство крутая госкорпора...,neutral
29233,С какими трудностями Вы сталкиваетесь как кайд...,оформл ведом премир,skip,оформление ведом премир,positive


In [890]:
df_all[(df_all['sentiment'] == 'skip') & (df_all['sentiment_corr'] != 'skip')]

,question,answer,count,sentiment,corrected,sentiment_corr
360,Ваша главная эмоции сейчас?,страшно чутка,1.0,skip,страшно немного,negative
1458,Как ваше настроение?,#росатомсила,1.0,skip,крутая госкорпорация росатом сила,positive
1622,Как ваше настроение?,бамбалейла,1.0,skip,отлично,positive
1973,Основная мысль по результатам конференции,росатом - люди,1.0,skip,крутая госкорпорация росатом - люди,neutral
2020,Основная мысль по результатам конференции,росатом - лидер,1.0,skip,крутая госкорпорация росатом - лидер,positive
...,...,...,...,...,...,...
29129,Какой ваш девиз,ВК - от Росатома пароль,1.0,skip,внешний контроль - от крутая госкорпорация рос...,neutral
29133,Какой ваш девиз,"ВК, к лидерству Росатома парол",1.0,skip,"вк, к лидерству крутая госкорпорация росатом п...",positive
29134,Какой ваш девиз,ВК к лидр Росатома пароль,1.0,skip,внешний контроль к лидерство крутая госкорпора...,neutral
29233,С какими трудностями Вы сталкиваетесь как кайд...,оформл ведом премир,1.0,skip,оформление ведом премир,positive


In [891]:
from spellchecker import SpellChecker

spell = SpellChecker(language='ru')

In [892]:
print(spell.candidates('оформл'))

{'формы', 'форма', 'форму', 'форме'}


In [893]:
df_all

,question,answer,count,sentiment,corrected,sentiment_corr
0,Что вы сможете использовать в своей работе?,"формулировать проблему, 5 поче",1.0,neutral,"формулировать проблему, 5 поче",neutral
1,Что вы сможете использовать в своей работе?,поиск потерь,1.0,neutral,поиск потерь,neutral
2,Что вы сможете использовать в своей работе?,правило 5 «почему» :),1.0,neutral,правило 5 «почему» :),neutral
3,Что вы сможете использовать в своей работе?,умение работать в команде,1.0,neutral,умение работать в команде,neutral
4,Что вы сможете использовать в своей работе?,картирование и анализ,1.0,neutral,картирование и анализ,neutral
...,...,...,...,...,...,...
29659,•Где вы хотите провести следующую HR-сессию?,в москве,1.0,neutral,в москве,neutral
29660,•Где вы хотите провести следующую HR-сессию?,в сочи,1.0,neutral,в сочи,neutral
29661,•Где вы хотите провести следующую HR-сессию?,сочи,1.0,neutral,сочи,neutral
29662,•Где вы хотите провести следующую HR-сессию?,окбм,1.0,neutral,окбм,neutral
